In [ ]:
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

dir = Path("/content/drive/MyDrive/Colab Notebooks/dataset_ventas") # directorio donde están los datasets (cambiar si hace falta)
# Leo los csv del directorio y los concateno en un dataframe
df = (pd.read_csv(f) for f in dir.glob("*.csv"))
df = pd.concat(df)
# Filtro el título
titulo = df.iloc[0]
# Filtra el DataFrame para eliminar las filas que coinciden con el título
df = df[df.apply(lambda row: not row.equals(titulo), axis=1)]
# Elimino los valores nulos del dataframe
df = df.dropna()
# Elimino las filas donde el ID no sea un número
df = df[df['ID de Pedido'].apply(lambda x: str(x).isdigit())]
# Restablece los índices si es necesario
df.reset_index(drop=True, inplace=True)
df.drop_duplicates()
df.describe()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio
count,185949,185949,185949,185949,185949,185949
unique,178436,19,9,23,142395,140786
top,160873,USB-C Charging Cable,1,11.95,12/15/19 20:16,"193 Forest St, San Francisco, CA 94016"
freq,5,21902,168552,21902,8,9


In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

df_copia = df
print(df_copia)

# Dividir la columna 'Dirección de Envío' en partes (Calle, Ciudad, Estado y Código Postal)
df_copia[['Calle', 'Ciudad', 'Estado CP']] = df['Dirección de Envio'].str.split(', ', n=2, expand=True)

# Dividir la columna 'Estado CP' en 'Estado' y 'Código Postal'
df_copia[['Estado', 'Código Postal']] = df['Estado CP'].str.split(' ', n=1, expand=True)

# Eliminar las columnas temporales 'Estado CP' y 'Dirección de Envío' si ya no las necesitas
df_copia.drop(columns=['Estado CP', 'Dirección de Envio'], inplace=True)


print(df_copia)
df_copia['Fecha de Pedido'] = pd.to_datetime(df_copia['Fecha de Pedido'])
mes = df_copia['Fecha de Pedido'].dt.month
df_copia['Cantidad Pedida'] = df_copia['Cantidad Pedida'].astype(int)
df_copia['Precio Unitario'] = df_copia['Precio Unitario'].astype(float)
df_copia['Monto Total'] = df_copia['Cantidad Pedida'] * df_copia['Precio Unitario']
# Agrupa los datos por mes y calcula la suma del monto total en cada mes
ventas_por_mes = df_copia.groupby(df_copia['Fecha de Pedido'].dt.month)['Monto Total'].sum().reset_index()
# Ordena el DataFrame por la columna de fecha
df_copia = df_copia.sort_values(by='Fecha de Pedido')
# Restablece los índices si es necesario
df_copia.reset_index(drop=True, inplace=True)
# La variable 'ventas_por_mes' contiene la suma de las ventas totales en cada mes
#print(df_copia)
# Muestra el resultado
#print(ventas_por_mes)
print("Mes con más ventas totales:", meses[ventas_por_mes['Monto Total'].idxmax()])
venta_mas_cara = df_copia['Monto Total'].idxmax()
print("Venta más cara en todo el año:\n", df_copia.iloc[venta_mas_cara])
# Agrupa los datos por día y calcula las ventas totales para cada día
df_copia['Ventas Totales'] = df_copia.groupby(df['Fecha de Pedido'].dt.date)['Monto Total'].transform('sum')
# Encuentra el día con las ventas totales más altas
dia_con_mas_ventas = df_copia[df_copia['Ventas Totales'] == df_copia['Ventas Totales'].max()]['Fecha de Pedido'].iloc[0].date()
dia_con_menos_ventas = df_copia[df_copia['Ventas Totales'] == df_copia['Ventas Totales'].min()]['Fecha de Pedido'].iloc[0].date()
# Imprime el resultado
print(f"Día con más ventas totales: {dia_con_mas_ventas}")
print(f"Día con menos ventas totales: {dia_con_menos_ventas}")
# Agrupa los datos por el nombre del producto y suma la cantidad vendida
productos_mas_vendidos = df.groupby('Producto')['Cantidad Pedida'].sum()
# Encuentra el producto más vendido
producto_mas_vendido = productos_mas_vendidos.idxmax()
cantidad_mas_vendida = productos_mas_vendidos.max()
# Imprime el resultado
print(f"El artículo más vendido es '{producto_mas_vendido}' con una cantidad vendida de {cantidad_mas_vendida}.")

# Extraer el mes de la fecha de pedido
df_copia['Mes'] = df_copia['Fecha de Pedido'].dt.month

# Agrupar por mes y producto, y calcular la cantidad total vendida en cada mes
ventas_por_mes_y_producto = df_copia.groupby(['Mes', 'Producto'])['Cantidad Pedida'].sum()

# Encontrar el producto más vendido en cada mes
productos_mas_vendidos_por_mes = ventas_por_mes_y_producto.groupby(level=0).idxmax()
print("Producto más vendido por mes:")
print(productos_mas_vendidos_por_mes)

       ID de Pedido                    Producto Cantidad Pedida  \
0            176559  Bose SoundSport Headphones               1   
1            176560                Google Phone               1   
2            176560            Wired Headphones               1   
3            176561            Wired Headphones               1   
4            176562        USB-C Charging Cable               1   
...             ...                         ...             ...   
185944       259353      AAA Batteries (4-pack)               3   
185945       259354                      iPhone               1   
185946       259355                      iPhone               1   
185947       259356      34in Ultrawide Monitor               1   
185948       259357        USB-C Charging Cable               1   

       Precio Unitario Fecha de Pedido  \
0                99.99  04/07/19 22:30   
1                  600  04/12/19 14:38   
2                11.99  04/12/19 14:38   
3                11.99  04/

In [ ]:
# Crear el objeto figura
fig = go.Figure()

# Agregar un gráfico de barras al objeto figura
fig.add_trace(go.Bar(
    #x=ventas_por_mes['Fecha de Pedido'],
    x=meses,
    y=ventas_por_mes['Monto Total'],
    text=ventas_por_mes['Monto Total'],
    marker=dict(
        color=ventas_por_mes['Monto Total'],
        colorscale='RdYlGn',
        colorbar=dict(title="Total de ventas")
    ),
    name='Ventas Totales'
))

# Configurar títulos y ejes del gráfico
fig.update_layout(
    title=dict(
        text='Ventas Totales por Mes',
        x=0.5  # Centrar el título
    ),
    xaxis_title='Meses',
    yaxis_title='Ventas Totales'
)

# Mostrar el gráfico
fig.show()

In [ ]:
ventas_por_hora = df_copia.groupby(df_copia['Fecha de Pedido'].dt.hour)['Cantidad Pedida'].sum()
print("Cantidad de ventas totales a una hora:\n", ventas_por_hora)
hora_max_ventas = ventas_por_hora.idxmax()
print("Hora donde más se vende:", hora_max_ventas)

Cantidad de ventas totales a una hora:
 Fecha de Pedido
0      4428
1      2619
2      1398
3       928
4       937
5      1493
6      2810
7      4556
8      7000
9      9816
10    12308
11    14005
12    14202
13    13685
14    12362
15    11391
16    11662
17    12229
18    13802
19    14470
20    13768
21    12244
22     9899
23     7065
Name: Cantidad Pedida, dtype: int64
Hora donde más se vende: 19


In [ ]:
import plotly.graph_objects as go

horas = list(range(25))

# Crear el objeto figura
fig = go.Figure()

# Agregar un gráfico de barras al objeto figura
fig.add_trace(go.Bar(
    x=horas,  # Horas en el eje X
    y=ventas_por_hora.values,  # Cantidad Pedida en el eje Y
    text=ventas_por_hora.values,
    marker=dict(
        color=ventas_por_hora.values,
        colorscale='RdYlGn',
        colorbar=dict(title="Total de ventas")
    ),
    name='Ventas por Hora'
))

# Configurar títulos y ejes del gráfico
fig.update_layout(
        title=dict(
        text='Cantidad de Ventas por Hora',
        x=0.5  # Centrar el título
    ),
    #title='Cantidad de Ventas por Hora',
    xaxis_title='Hora del día',
    yaxis_title='Ventas Totales',
    xaxis=dict(
        tickmode='linear',  # Modo de marcado lineal
        tick0=0,  # Valor inicial
        dtick=1  # Intervalo de marcado (en este caso, 1 hora)
    )
)

# Mostrar el gráfico
fig.show()

In [ ]:
# Agrupar por ciudad y calcular las ventas totales
ventas_por_ciudad = df_copia.groupby('Ciudad')['Monto Total'].sum()

# Ordenar las ciudades en orden descendente según las ventas totales
ciudades_en_orden_descendente = ventas_por_ciudad.sort_values(ascending=False).index.tolist()

# Imprimir la lista de ciudades en orden descendente
print("Ciudades en orden descendente de ventas totales:")
print(ciudades_en_orden_descendente)

# Agrupar por estado y calcular las ventas totales
ventas_por_estado = df_copia.groupby('Estado')['Monto Total'].sum()

# Ordenar las ciudades en orden descendente según las ventas totales
estados_en_orden_descendente = ventas_por_estado.sort_values(ascending=False).index.tolist()

# Imprimir la lista de ciudades en orden descendente
print("Estados en orden descendente de ventas totales:")

def obtener_nombres_estados(codigos_estados):
    estados = {
        'CA': 'California',
        'NY': 'New York',
        'TX': 'Texas',
        'MA': 'Massachusetts',
        'GA': 'Georgia',
        'WA': 'Washington',
        'OR': 'Oregon',
        'ME': 'Maine'
        # Agrega más estados según sea necesario
    }

    # Utilizar comprensión de lista para obtener nombres de estado para cada código
    nombres_estados = [estados.get(codigo, 'Estado Desconocido') for codigo in codigos_estados]

    return nombres_estados

print(obtener_nombres_estados(estados_en_orden_descendente))

Ciudades en orden descendente de ventas totales:
['San Francisco', 'Los Angeles', 'New York City', 'Boston', 'Atlanta', 'Dallas', 'Seattle', 'Portland', 'Austin']
Estados en orden descendente de ventas totales:
['California', 'New York', 'Texas', 'Massachusetts', 'Georgia', 'Washington', 'Oregon', 'Maine']


In [ ]:
# Extraer el día de la semana (lunes, martes, miércoles, etc.) de la fecha de pedido
df_copia['Día de la Semana'] = df_copia['Fecha de Pedido'].dt.day_name()

# Agrupar por día de la semana y calcular las ventas totales para cada día
ventas_por_dia_semana = df_copia.groupby('Día de la Semana')['Monto Total'].sum()

# Ordenar los días de la semana en el orden deseado (por ejemplo, de lunes a domingo)
# Ordenar los días de la semana en el orden deseado (por ejemplo, de lunes a domingo)
dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
ventas_por_dia_semana = ventas_por_dia_semana[dias_ordenados]

# Crear un gráfico de barras con Plotly
fig = go.Figure()
fig.add_trace(go.Bar(
    x=ventas_por_dia_semana.index,
    y=ventas_por_dia_semana.values,
    marker=dict(color='blue'),  # Color de las barras
))
fig.update_layout(
    title='Variación de Ventas a lo largo de la Semana',
    xaxis_title='Día de la Semana',
    yaxis_title='Ventas Totales',
    xaxis=dict(tickangle=45),  # Rotar etiquetas del eje x para una mejor visualización
)
fig.show()

In [ ]:
# Lista de fechas de días festivos (puedes personalizar esto según tus datos)
dias_festivos = ['2023-01-01', '2023-07-04', '2023-12-25']

# Definir un período de tiempo antes y después de los eventos especiales (por ejemplo, 3 días antes y después)
dias_antes = 3
dias_despues = 3

# Calcular las ventas totales para cada período alrededor de los días festivos
ventas_alrededor_de_festivos = []

for festivo in dias_festivos:
    fecha_festivo = pd.to_datetime(festivo)
    inicio_periodo = fecha_festivo - pd.DateOffset(days=dias_antes)
    fin_periodo = fecha_festivo + pd.DateOffset(days=dias_despues)

    ventas_periodo = df_copia[(df_copia['Fecha de Pedido'] >= inicio_periodo) & (df_copia['Fecha de Pedido'] <= fin_periodo)]['Monto Total'].sum()

    ventas_alrededor_de_festivos.append((festivo, ventas_periodo))

# Imprimir las ventas alrededor de los días festivos
for festivo, ventas in ventas_alrededor_de_festivos:
    print(f"Fecha del festivo: {festivo}, Ventas totales alrededor del festivo: {ventas}")

Fecha del festivo: 2023-01-01, Ventas totales alrededor del festivo: 0.0
Fecha del festivo: 2023-07-04, Ventas totales alrededor del festivo: 0.0
Fecha del festivo: 2023-12-25, Ventas totales alrededor del festivo: 0.0
